### 6강 Policy Iteration

안녕하세요, 학생 여러분! 오늘 우리는 인공지능 에이전트가 게임을 학습하는 또 다른 방법, 바로 Policy Iteration에 대해 배워보겠습니다.

이번 강의에서는 FrozenLake-v1 환경을 사용하여 Policy Iteration 알고리즘을 적용해볼 것입니다. FrozenLake는 에이전트가 시작 지점(S)에서 목적지(G)까지 도달해야 하는, 얼음(H)과 구멍(F)이 있는 격자 환경입니다. 에이전트는 목적지에 도달하기 위해 안전한 경로를 찾아야 하며, 얼음 위를 걷다가 구멍에 빠지면 게임이 종료됩니다.

Policy Iteration은 강화학습의 중요한 방법 중 하나로, 주어진 환경에서 최적의 정책을 찾아가는 과정입니다. 이 방법은 크게 두 단계로 나뉩니다: 첫 번째는 현재 정책에 대한 가치 함수를 평가하는 정책 평가(Policy Evaluation) 단계와, 두 번째는 가치 함수를 바탕으로 정책을 개선하는 정책 개선(Policy Improvement) 단계입니다.

Policy Evaluation
정책 평가 단계에서는 현재 정책 하에 각 상태의 가치를 계산합니다. 이를 통해 어떤 상태에서 어떤 행동을 취하는 것이 장기적으로 얼마나 가치가 있는지 평가할 수 있습니다.

Policy Improvement
가치 함수를 계산한 후, 이 정보를 사용하여 각 상태에서 더 좋은 행동을 선택하는 새로운 정책을 만듭니다. 이 과정을 반복하면, 점차 최적의 정책에 도달하게 됩니다.

FrozenLake 환경에서 Policy Iteration을 적용함으로써, 우리는 에이전트가 어떻게 학습하고 최적의 경로를 찾아가는지 관찰할 수 있습니다. 이는 강화학습에서 정책을 직접 평가하고 개선하는 방법의 효율성을 이해하는 데 도움이 될 것입니다.

자, 그럼 FrozenLake-v1 환경에서 Policy Iteration을 적용해보며, 이 강력한 강화학습 기법이 실제로 어떻게 작동하는지 함께 살펴보도록 하겠습니다. 이 과정을 통해, 강화학습의 핵심 개념들과 이를 통한 문제 해결 방법에 대한 깊은 이해를 얻을 수 있을 것입니다.


# 1.Step 1 설치

필요한 라이브러리 설치
주피터 노트북에서는 `!`를 사용하여 터미널 명령을 실행할 수 있습니다.

In [7]:
!pip3 install --upgrade gym
!pip3 install gym_super_mario_bros nes_py
!pip3 install gym

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


# Step 2 필요한 라이브러리 임포트


gym: 강화학습 환경을 제공하는 라이브러리
gym_super_mario_bros: Super Mario Bros 게임 환경
JoypadSpace, SIMPLE_MOVEMENT: 조작과 행동을 단순화하기 위한 도구들

In [8]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import gym


In [11]:
import gymnasium as gym
import numpy as np

def policy_evaluation(policy, env, discount_factor=0.99, theta=1e-10):  # Discount factor와 theta 값 조정
    value_table = np.zeros(env.observation_space.n)
    while True:
        delta = 0
        for state in range(env.observation_space.n):
            v = 0
            for action in range(env.action_space.n):
                for prob, next_state, reward, done in env.P[state][action]:
                    action_prob = policy[state][action]
                    v += action_prob * prob * (reward + discount_factor * value_table[next_state])
            delta = max(delta, np.abs(value_table[state] - v))
            value_table[state] = v
        if delta < theta:
            break
    return value_table

def policy_improvement(env, value_table, discount_factor=0.99):  # Discount factor 조정
    policy = np.zeros([env.observation_space.n, env.action_space.n])
    for state in range(env.observation_space.n):
        q_values = np.zeros(env.action_space.n)
        for action in range(env.action_space.n):
            for prob, next_state, reward, done in env.P[state][action]:
                q_values[action] += prob * (reward + discount_factor * value_table[next_state])
        best_action = np.argmax(q_values)
        policy[state, :] = 0
        policy[state, best_action] = 1.0
    return policy

env = gym.make('FrozenLake-v1', is_slippery=True, render_mode="human")
policy = np.ones([env.observation_space.n, env.action_space.n]) / env.action_space.n
iterations = 0
env.reset()

while True:
    value_table = policy_evaluation(policy, env)
    new_policy = policy_improvement(env, value_table)
    if np.array_equal(new_policy, policy):
        break
    policy = new_policy
    iterations += 1

env.render()
print(f"Policy optimized after {iterations} iterations")
print("Optimized Policy:")
print(policy)
env.close()


Policy optimized after 2 iterations
Optimized Policy:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]
